In [1]:
import pandas as pd
import sqlite3 as sql
import pandas as pd
from geopy.geocoders import ArcGIS
import time
import shapely.geometry as geom
import geopandas as gpd

In [2]:
data = pd.read_csv("C:\Users\Graham\Documents\STA 141C\project\police_department_incidents.csv")

In [3]:
data['index_right'] = range(len(data))

In [4]:
# LOAD IN ZIP CODES
zipshapes = gpd.read_file("C:\Users\Graham\Documents\STA 141C\project\cb_2015_us_zcta510_500k/cb_2015_us_zcta510_500k.shp")
zipshapes.head()

# NARROW IT DOWN SOME
zipshapes = zipshapes[zipshapes.ZCTA5CE10.str.startswith("9")]

#CUT OUT UNNESSESARY STUFF
zipshapes = zipshapes[['geometry', 'ZCTA5CE10']]

In [5]:
points = gpd.GeoDataFrame([geom.asPoint(tuple(row)) for row in data[["X","Y"]].values], columns=["geometry"])

In [6]:
zipcodes = gpd.sjoin(zipshapes, points, how="inner", op="contains")

In [8]:
newdata = data.merge(zipcodes, on=['index_right'])

In [53]:
newdata['zipcode'] = [str(x) for x in newdata['ZCTA5CE10']]

KeyError: 'ZCTA5CE10'

In [11]:
action = ['ARREST, BOOKED', 'ARREST, CITED', 'JUVENILE BOOKED', 'PROSECUTED BY OUTSIDE AGENCY', 'JUVENILE CITED', 'JUVENILE DIVERTED'
         , 'PROSECTUTED FOR LESSER OFFENSE']

In [12]:
action_taken = [x in action for x in newdata['Resolution']]
newdata['action_taken'] = action_taken

In [47]:
newdata = newdata.drop('index_right', axis=1)
newdata = newdata.drop('ZCTA5CE10', axis=1)
newdata = newdata.drop('PdId', axis=1)

ValueError: labels ['index_right'] not contained in axis

In [13]:
db = sql.connect('C:\Users\Graham\Documents\STA 141C\project\sf_data.sqlite')

In [14]:
cursor = db.execute("SELECT * FROM sqlite_master")

In [15]:
zillow = pd.read_sql("SELECT * from zillow", db)

In [51]:
#newdata.merge(zillow, how='inner', on='RegionName')